GRU Model 

In [ ]:
#  algorithms/gru_model.py logic in‐notebook

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

class GRUTrafficPredictor:
    def __init__(self, data: pd.DataFrame, sequence_length: int = 96, epochs: int = 30):
        """
        data: DataFrame with ['Timestamp','Site_ID','Volume']
        sequence_length: number of 15‐min intervals in your input window (default 96 = 24 h)
        epochs: how many epochs to train
        """
        # 1) Sort by time
        self.df = data.sort_values("Timestamp").reset_index(drop=True)
        self.seq_len = sequence_length
        self.epochs  = epochs

        # 2) extract & scale volumes
        ts = pd.to_datetime(self.df["Timestamp"])
        vols = self.df["Volume"].values.reshape(-1,1)
        self.scaler       = MinMaxScaler(feature_range=(0,1))
        self.vols_scaled  = self.scaler.fit_transform(vols)
        self.timestamps   = ts.values

        # placeholder for the trained model
        self.model = None

    def _make_sequences(self, series: np.ndarray):
        X, y = [], []
        for i in range(len(series) - self.seq_len):
            X.append(series[i : i + self.seq_len])
            y.append(series[i + self.seq_len, 0])
        return np.array(X), np.array(y)

    def train_model(self):
        # 1) build (X,y) from the entire history
        X, y = self._make_sequences(self.vols_scaled)

        # 2) split chronologically
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, shuffle=False
        )

        # 3) defines a 2‐layer GRU
        self.model = tf.keras.Sequential([
            tf.keras.Input(shape=(self.seq_len, 1)),
            tf.keras.layers.GRU(64, return_sequences=True, activation="tanh"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.GRU(32, activation="tanh"),
            tf.keras.layers.Dense(1),
        ])
        self.model.compile(optimizer="adam", loss="mse")

        # 4) train with early stopping
        self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=self.epochs,
            batch_size=64,
            callbacks=[tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=3, restore_best_weights=True
            )],
            verbose=2
        )


train.py

In [ ]:
# train & persist weights 


import os
import pandas as pd


# 1) Load your processed CSV
csv_path = r"C:\Users\miran\Downloads\traffic_model_ready.csv"
print(f"📥 Loading data from {csv_path} …")
df = pd.read_csv(csv_path, parse_dates=["Timestamp"])
df = df.sort_values("Timestamp").reset_index(drop=True)

# 2) Train the GRU
predictor = GRUTrafficPredictor(
    data=df,
    sequence_length=96,  # 96×15 min = 24 h
    epochs=10            
)
print("▶️  Training GRU model…")
predictor.train_model()

# 3) Save the full model (architecture + weights) into Downloads
download_dir = r"C:\Users\miran\Downloads"
os.makedirs(download_dir, exist_ok=True)
out_path = os.path.join(download_dir, "gru_model.h5")
predictor.model.save(out_path)
print(f"✅  Saved GRU model to {out_path}")


📥 Loading data from C:\Users\miran\Downloads\traffic_model_ready.csv …
▶️  Training GRU model…
Epoch 1/10
5030/5030 - 635s - 126ms/step - loss: 0.0065 - val_loss: 0.0060
Epoch 2/10
5030/5030 - 561s - 112ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 3/10
5030/5030 - 534s - 106ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 4/10
5030/5030 - 621s - 123ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 5/10
5030/5030 - 533s - 106ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 6/10
5030/5030 - 600s - 119ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 7/10
5030/5030 - 596s - 118ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 8/10
5030/5030 - 511s - 102ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 9/10
5030/5030 - 563s - 112ms/step - loss: 0.0063 - val_loss: 0.0059


✅  Saved GRU model to C:\Users\miran\Downloads\gru_model.h5
